# Database Queries


This is the first notebook in a series of Jupyter Notebooks linked to the project titled "__Developing Mechanism-Based Models for Complex Toxicology Study Endpoints Using Standardized Electronic Submission Data__".  They should be somewhat ordinal, i.e., should be run in order as each Notebook could depend on results from a previous notebook.  

This notebook requires a dependency script, `send.py`, which contains an object that is a live instance to the database and can be queried, etc.  This script requires two variables to be configured, `db_dir` and `send_db_file`.  `db_dir` should point to the directory that contains the SQLlite database and `send_db_file` should be the SQLite database file itself. 

This notebook simply is simply just to get database queries for the SEND manuscript.


In [104]:
from send import send_db
import pandas as pd

ts = send_db.generic_query('SELECT * FROM TS')

In [105]:
print(f"There are {ts.TSPARMCD.nunique()} TSPARMCDs")
print('\n'.join(ts.TSPARMCD.unique()))

There are 101 TSPARMCDs
AGETXT
AGEU
DOSDUR
EXPENDTC
EXPSTDTC
GLPTYP
HOUSEGRP
RECSAC
ROUTE
SDESIGN
SNDCTVER
SNDIGVER
SPECIES
SPLRLOC
SPLRNAM
SPREFID
SSPONSOR
SSTYP
STCAT
STDIR
STITLE
STRAIN
STSTDTC
TFCNTRY
TRMSAC
TRT
TRTCAS
TRTV
TSTFLOC
TSTFNAM
ASOCSTDY
BEDCHNG
BEDDING
DIET
DOSENDTC
DOSSTDTC
ENVTEMP
ENVTEMPU
FEEDREG
GLPFL
HOUSETYP
HUMIDT
HUMIDTU
IACUC
IDMETH
LIGHT
MTHTRM
PCLASS
SEXPOP
SRANDOM
STENDTC
TRTUNII
WATER
WTRDLVRY
STMON
PINV
TRTDOSU
SLENGTH
PLANFSUB
PLANMSUB
SPLANSUB
TSCNTRY
TSLOC
TSNAM
TKDESC
AGE
PPL
PDOSFRQ
CNTRBSC
STRPSTAT
SBSTRAIN
TOTANPCH
ALTSTDID
INTSAC
PPTCNAM
PPTEGID
PPTEGSYM
PPTMDA
STOBJ
STRPNUM
TANUM
AMGPNUM
GPSNUM
THAGTMOD
QATYPE
PROTNUM
STRPDT
PREPPER
PDFORM
SPFIND
LOT
TESTID
TCNTRL
PKANIND
DEFVER
DOSFRQ
PCLAS
PRJCTID
CLASS
TELMIND
WATRDLVRY


In [106]:
animals = send_db.get_all_animals()

animals.head()

,STUDYID,ID,USUBJID,SPECIES,STRAIN,SEX
0,00086621,1,00086621-1001,DOG,BEAGLE,M
1,00086621,1,00086621-1002,DOG,BEAGLE,M
2,00086621,1,00086621-1003,DOG,BEAGLE,M
3,00086621,1,00086621-2001,DOG,BEAGLE,M
4,00086621,1,00086621-2002,DOG,BEAGLE,M


### Results section 3.1

In [107]:
animals.USUBJID.nunique()

169839

In [108]:
animals.SPECIES.value_counts()

RAT           95464
MOUSE         42770
MONKEY        16980
DOG           11629
PIG            1657
RABBIT         1426
GUINEA PIG      334
CAT             178
MINIPIG           2
Name: SPECIES, dtype: int64

In [109]:
95464/169839

0.5620852689900435

In [110]:
training = pd.read_csv('data/RAT/RAT_training_data.csv')

/hive/Anaconda3-2020.02/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [111]:
training.USUBJID.nunique()

17285

### Results section 3.2

In [112]:
mi = send_db.generic_query('SELECT * FROM MI')

mi['MISPEC'] = mi['MISPEC'].str.upper()
mi['MISTRESC'] = mi['MISTRESC'].str.upper()

liver = mi[mi.MISPEC == 'LIVER']

In [113]:
print(mi.MISPEC.value_counts())
print(mi.shape[0])

LIVER                                     114636
KIDNEY                                    105560
GLAND, ADRENAL                             97592
SPLEEN                                     89112
THYMUS                                     87497
                                           ...  
ESOPHAGUS/TRACHEA                              1
LYMPH NODE,  MESENTERIC                        1
DOSING SITE #1                                 1
BRAIN, CEREBRUM/BRAIN, CEREBELLUM              1
HEMOLYMPHORETICULAR SYSTEM (ALL SITES)         1
Name: MISPEC, Length: 579, dtype: int64
3803024


In [114]:
liver_training_rats = liver[liver.USUBJID.isin(training.USUBJID.values)]

print(liver_training_rats.head())

     STUDYID DOMAIN        USUBJID  MISEQ  ... FOCID MICHRON MIDISTR VISITDY
29   8391504     MI  8391504-R0001     30  ...  None    None    None     NaN
30   8391504     MI  8391504-R0001     31  ...  None    None    None     NaN
57   8391504     MI  8391504-R0002     58  ...  None    None    None     NaN
118  8391504     MI  8391504-R0003    119  ...  None    None    None     NaN
175  8391504     MI  8391504-R0004    176  ...  None    None    None     NaN

[5 rows x 32 columns]


In [115]:
liver_training_rats.MISTRESC.value_counts().to_csv('data/RAT/text/training_set_liver_findings.csv')

## Results Section 3.3

In [116]:
def filter_text(x):
    """ returns null if x does not contain a valid numeric response,
    else it extracts that using a regex pattern """
    digit_pattern = r'[-+]?([0-9]*\.[0-9]+|[0-9]+)'
    digit_extract = re.search(digit_pattern, str(x))
    if digit_extract:
        return float(digit_extract.group(0))
    return np.nan


In [117]:
lb = send_db.generic_query('SELECT * FROM LB')

In [118]:
lb_training_rats =  lb[lb.USUBJID.isin(training.USUBJID.values)]
lb_training_rats.LBTESTCD = lb_training_rats.LBTESTCD.str.upper()
lb_training_rats.LBSPEC = lb_training_rats.LBSPEC.str.upper()
lb_training_rats.loc[:, 'LBSTRESC'] = lb_training_rats.LBSTRESC.apply(filter_text)
lb_training_rats = lb_training_rats[lb_training_rats.LBSTRESC.notnull()

SyntaxError: unexpected EOF while parsing (<ipython-input-118-34faa4eb5769>, line 5)

In [ ]:
unique_pairs = lb_training_rats[['LBTESTCD', 'LBSPEC']].drop_duplicates()

In [ ]:
no_tests = sum(['-' in test for test in training.columns])

print(f"There are {no_tests} tests in the training set.")

In [120]:
training

,Unnamed: 0,STUDYID,USUBJID,A2MACG-SERUM,ABCELL-WHOLE BLOOD,ABNCE-URINE,ACANT-WHOLE BLOOD,ACETOAC-PLASMA,ACTH-PLASMA,ACYTTCEL-WHOLE BLOOD,ADA-SERUM,ADPNCTN-PLASMA,ADPNCTN-SERUM,AHLPTCEL-WHOLE BLOOD,ALB-PLASMA,ALB-SERUM,ALB-SERUM CHEMISTRY,ALB-UNSPECIFIED,ALB-URINE,ALB-WHOLE BLOOD,ALBEXR-URINE,ALBGLOB-PLASMA,ALBGLOB-SERUM,ALBGLOB-SERUM CHEMISTRY,ALBGLOB-UNSPECIFIED,ALBGLOB-WHOLE BLOOD,ALBPROT-SERUM,ALBPROT-UNSPECIFIED,ALDOLASE-SERUM,ALP-PLASMA,ALP-SERUM,ALP-SERUM CHEMISTRY,ALP-UNSPECIFIED,ALP-WHOLE BLOOD,ALPIS-PLASMA,ALT-PLASMA,ALT-SERUM,ALT-SERUM CHEMISTRY,ALT-UNSPECIFIED,ALT-WHOLE BLOOD,...,UREA-PLASMA,UREA-SERUM,UREA-UNSPECIFIED,UREA-WHOLE BLOOD,UREACRT-PLASMA,UREAN-PLASMA,UREAN-SERUM,UREAN-UNSPECIFIED,UREAN-WHOLE BLOOD,UREANCRT-SERUM,UROBIL-UNSPECIFIED,UROBIL-URINE,UROTHC-URINE,UVOL1-URINE,UVOL2-URINE,UVOLT-URINE,VEGF-SERUM,VITK1-PLASMA,VITK1O-PLASMA,VOLUME-UNSPECIFIED,VOLUME-URINE,WBC-UNSPECIFIED,WBC-URINE,WBC-WHOLE BLOOD,WBCCLMP-URINE,YEAST-UNSPECIFIED,YEAST-URINE,MISTRESC,STEATOSIS,CHOLESTASIS,NECROSIS,SPECIES,SEX,IS_CONTROL,BWDIFF,BWSLOPE,BWINTCEPT,BWDIFF_NORM,BWSLOPE_NORM,BWINTCEPT_NORM
0,0,0436RA140_001,0436RA140_001-4201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.996885,NaN,NaN,NaN,NaN,NaN,NaN,0.970874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.892330,NaN,NaN,NaN,NaN,NaN,0.902613,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.055276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.034483,1.300590,NaN,NaN,NaN,NORMAL,0.0,0.0,0.0,RAT,M,True,76.0,3.246941,262.966630,1.017403,1.046255,1.054913
1,1,0436RA140_001,0436RA140_001-4202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.965732,NaN,NaN,NaN,NaN,NaN,NaN,0.970874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.892330,NaN,NaN,NaN,NaN,NaN,0.736342,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.904523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.344828,0.891804,NaN,NaN,NaN,NORMAL,0.0,0.0,0.0,RAT,M,True,84.0,3.390434,262.122914,1.124498,1.092493,1.051529
2,2,0436RA140_001,0436RA140_001-4203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.028037,NaN,NaN,NaN,NaN,NaN,NaN,0.970874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.143068,NaN,NaN,NaN,NaN,NaN,0.973872,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.055276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.344828,1.139923,NaN,NaN,NaN,NORMAL,0.0,0.0,0.0,RAT,M,True,65.0,2.672970,247.341491,0.870147,0.861306,0.992232
3,3,0436RA140_001,0436RA140_001-4204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.965732,NaN,NaN,NaN,NaN,NaN,NaN,0.970874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.157817,NaN,NaN,NaN,NaN,NaN,1.187648,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.854271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.034483,0.942648,NaN,NaN,NaN,NORMAL,0.0,0.0,0.0,RAT,M,True,55.0,2.256396,221.978865,0.736278,0.727074,0.890487
4,4,0436RA140_001,0436RA140_001-4205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.965732,NaN,NaN,NaN,NaN,NaN,NaN,0.970874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.157817,NaN,NaN,NaN,NaN,NaN,0.783848,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.055276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.034483,0.644702,NaN,NaN,NaN,NORMAL,0.0,0.0,0.0,RAT,M,True,59.0,2.487764,252.866518,0.789826,0.801627,1.014396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17280,17280,YV92WD,YV92WD-4637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.761905,NaN,NaN,NaN,NaN,NaN,NaN,0.865385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.846287,NaN,NaN,NaN,NaN,NaN,0.947368,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.021898,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.527778,NaN,NaN,1.686716,NaN,NaN,NaN,EXTRAMEDULLARY HEMATOPOIESIS,0.0,0.0,0.0,RAT,F,False,39.3,1.562973,230.345676,0.908460,1.031875,0.943198
17281,17281,YV92WD,YV92WD-4638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [127]:
# Define species to make a training set
# and make a seperate folder to store
# all the resulting data
import os, numpy as np, math

species = 'RAT'

   
species_data = os.path.join('data', species)
model_folder = os.path.join(species_data, 'models')
prediction_folder = os.path.join(species_data, 'predictions')
    
training_data_file = os.path.join(species_data, f'{species}_training_data.csv')

In [129]:
min_response_value = 0.4

df = pd.read_csv(training_data_file, index_col=0)
df = df.replace(np.inf, np.nan)
srted_tests = df.notnull().sum().sort_values(ascending=False)

good_tests = df.columns[(df.notnull().sum() / df.shape[0]) > min_response_value]
good_tests = good_tests[~good_tests.isin(['USUBJID', 'STUDYID', 'SEX', 'STEATOSIS',
                                         'CHOLESTASIS', 'NECROSIS', 'SPECIES', 'IS_CONTROL',
                                         'BWDIFF', 'BWSLOPE', 'BWINTCEPT', 'MISTRESC'])]

data = df[good_tests]
data = data.apply(lambda x: x + abs(x.min()) + 1)
data = data.applymap(math.log10)


data.index = df.USUBJID
data['SEX'] = df['SEX']



In [147]:
tests = []

for t in data.columns:
    
    if '-' in t:
        test, spec = t.split('-')
        
        desc = lb[lb.LBTESTCD == test].LBTEST.value_counts().index[0]
        tests.append([test, spec, desc])
        
    else:
        tests.append([t, 'NA', ''])
tests = pd.DataFrame(tests)
tests.columns = ['Test', 'Tissue', 'Description']

Albumin
Albumin/Globulin
Alkaline Phosphatase
Alanine Aminotransferase
Activated Partial Thromboplastin Time
Aspartate Aminotransferase
Basophils
Bilirubin
Calcium
Cholesterol
Creatine Kinase
Chloride
Creatinine
Eosinophils
Fibrinogen
Gamma Glutamyl Transferase
Globulin
Glucose
Hematocrit
Hemoglobin
Potassium
Large Unstained Cells
Lymphocytes
Ery. Mean Corpuscular Hemoglobin
Ery. Mean Corpuscular HGB Concentration
Ery. Mean Corpuscular Volume
Monocytes
Neutrophils
pH
Phosphate
Platelets
Protein
Protein
Prothrombin Time
Erythrocytes
Erythrocytes Distribution Width
Reticulocytes
Reticulocytes/Erythrocytes
Sodium
Specific Gravity
Triglycerides
Urea Nitrogen
Volume
Leukocytes


In [148]:
tests.to_csv('data/RAT/text/tests.csv')

In [143]:
lb

,STUDYID,DOMAIN,USUBJID,POOLID,LBSEQ,LBGRPID,LBREFID,LBSPID,LBTESTCD,LBTEST,LBCAT,LBSCAT,LBORRES,LBORRESU,LBORNRLO,LBORNRHI,LBSTRESC,LBSTRESN,LBSTRESU,LBSTNRLO,LBSTNRHI,LBSTNRC,LBNRIND,LBSTAT,LBREASND,LBNAM,LBSPEC,LBANTREG,LBSPCCND,LBSPCUFL,LBLOC,LBLAT,LBDIR,LBPORTOT,LBMETHOD,LBBLFL,LBFAST,LBDRVFL,LBTOX,LBTOXGR,LBEXCLFL,LBREASEX,VISITDY,LBDTC,LBENDTC,LBDY,LBENDY,LBTPT,LBTPTNUM,LBELTM,LBTPTREF,LBRFTDTC,LBUSCHFL,LBNOMDY,LBNOMLBL
0,8391504,LB,8391504-R0001,None,1,2573481,2573481,None,ALT,Alanine Aminotransferase,CLINICAL CHEMISTRY,None,45,U/L,None,None,45,45.00,U/L,None,None,None,None,,,None,SERUM,None,,None,Jugular Vein,None,None,None,Cobas 8000 Asset: 019456-Madison:Alanine Amino...,,None,None,None,None,None,None,47.0,2018-10-15T07:15:24,None,47.0,NaN,Clin Path,1.0,None,None,None,None,NaN,None
1,8391504,LB,8391504-R0001,None,2,2573481,2573481,None,ALP,Alkaline Phosphatase,CLINICAL CHEMISTRY,None,62,U/L,None,None,62,62.00,U/L,None,None,None,None,,,None,SERUM,None,,None,Jugular Vein,None,None,None,Cobas 8000 Asset: 019456-Madison:Alkaline Phos...,,None,None,None,None,None,None,47.0,2018-10-15T07:15:24,None,47.0,NaN,Clin Path,1.0,None,None,None,None,NaN,None
2,8391504,LB,8391504-R0001,None,3,2573481,2573481,None,CA,Calcium,CLINICAL CHEMISTRY,None,10.8,mg/dL,None,None,10.8,10.80,mg/dL,None,None,None,None,,,None,SERUM,None,,None,Jugular Vein,None,None,None,Cobas 8000 Asset: 019456-Madison:Calcium,,None,None,None,None,None,None,47.0,2018-10-15T07:15:24,None,47.0,NaN,Clin Path,1.0,None,None,None,None,NaN,None
3,8391504,LB,8391504-R0001,None,4,2573481,2573481,None,PHOS,Phosphate,CLINICAL CHEMISTRY,None,6.6,mg/dL,None,None,6.6,6.60,mg/dL,None,None,None,None,,,None,SERUM,None,,None,Jugular Vein,None,None,None,Cobas 8000 Asset: 019456-Madison:Inorganic Pho...,,None,None,None,None,None,None,47.0,2018-10-15T07:15:24,None,47.0,NaN,Clin Path,1.0,None,None,None,None,NaN,None
4,8391504,LB,8391504-R0001,None,5,2573481,2573481,None,GLUC,Glucose,CLINICAL CHEMISTRY,None,84,mg/dL,None,None,84,84.00,mg/dL,None,None,None,None,,,None,SERUM,None,,None,Jugular Vein,None,None,None,Cobas 8000 Asset: 019456-Madison:Glucose,,None,None,None,None,None,None,47.0,2018-10-15T07:15:24,None,47.0,NaN,Clin Path,1.0,None,None,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11285698,1322-19047,LB,1322-19047-3F060,,3114,,147554,,SODIUM,Sodium,Clinical Chemistry,,144,mEq/L,None,None,144,144.00,mEq/L,None,None,,,,,,,,,,,,,,Advia 1800,,,,,,,,8.0,2019-03-25,,8.0,NaN,,NaN,,,,None,NaN,None
11285699,1322-19047,LB,1322-19047-3F060,,3115,,147554,,CL,Chloride,Clinical Chemistry,,104,mEq/L,None,None,104,104.00,mEq/L,None,None,,,,,,,,,,,,,,Advia 1800,,,,,,,,8.0,2019-03-25,,8.0,NaN,,NaN,,,,None,NaN,None
11285700,1322-19047,LB,1322-19047-3F060,,3116,,147614,,CLARITY,Clarity,Urinalysis,,clear,,None,None,clear,NaN,,None,None,,,,,,,,,,,,,,,,,,,,,,8.0,2019-03-25,,8.0,NaN,,NaN,,,,None,NaN,None
11285701,1322-19047,LB,1322-19047-3F060,,3117,,147554,,K,Potassium,Clinical Chemistry,,9.7,mEq/L,None,None,9.7,9.70,mEq/L,None,None,,,,,,,,,,,,,,Advia 1800,,,,,,,,8.0,2019-03-25,,8.0,NaN,,NaN,,,,None,NaN,None


In [149]:
cl = send_db.generic_query('SELECT * FROM CL')

In [152]:
cl.CLSTRESC.str.upper().value_counts().index.tolist()

['NORMAL',
 'HAIR SPARSE',
 '0',
 'ERYTHEMA',
 'EDEMA',
 'SKIN AND PELAGE:THINNING HAIR COAT',
 'FUR, THIN COVER',
 'THICKENING',
 'ALOPECIA',
 'SKIN, RED',
 'HAIR LOSS',
 'SKIN - BRUISES',
 'SCAB',
 'SKIN AND PELAGE:DISCOLORED HAIRCOAT',
 'HAIR DISCOLORED',
 'SKIN DISCOLORED',
 'SCABBED AREA',
 'SALIVATION',
 'EXCRETION:FECES: NORMAL (1.OBSERVATION)',
 'FECES, SOFT',
 'SWELLING',
 'QUALITATIVE FOOD CONSUMPTION:FOOD CONSUMPTION: NORMAL (1.OBSERVATION)',
 'BRUISE',
 'SKIN, BRUISE',
 'MATERIAL AROUND EYES',
 'SKIN RED',
 'FECES SOFT',
 'BRUISING',
 'SKIN DRY',
 'FUR THIN',
 'SKIN, SCAB',
 'THIN',
 'FUR STAINED RED',
 'ABRASION',
 'SKIN, DRY',
 'ABNORMAL COLOR',
 'NO VISIBLE LESIONS:NO VISIBLE LESIONS',
 'SKIN AND PELAGE:SCAB',
 'HAIRCOAT, THIN',
 'ACTIVITY DECREASED',
 'FUR, STAINING, YELLOW',
 'SWOLLEN',
 'FECES WATERY',
 'HAIR:HAIR LOSS',
 'THIN COAT',
 'GOOD',
 'NODULE',
 'ABNORMAL URINE COLOR',
 'ABSENT',
 'NAD',
 'EXCRETION:FECES, NONFORMED',
 'MATERIAL PRESENT',
 'NORMAL FOOD CONSU

In [153]:
training.shape

(17285, 641)

In [154]:
training.IS_CONTROL.sum()

4481

In [157]:
ts_training = ts[ts.STUDYID.isin(training.STUDYID)]

In [156]:
17285-4481

12804

In [169]:
vals = ts_training[ts_training.TSPARMCD == 'ROUTE'][['STUDYID', 'TSVAL']].drop_duplicates()

In [170]:
ts_training.STUDYID.unique().shape

(138,)

In [180]:
vals.value_counts()

AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [181]:
vals.TSVAL.value_counts()

ORAL GAVAGE                 93
INTRAVENOUS                 19
INTRAVENOUS BOLUS           12
SUBCUTANEOUS                 8
RESPIRATORY (INHALATION)     5
ORAL                         3
INTRAVENOUS DRIP             2
INTRAMUSCULAR                1
INTRAVITREAL                 1
INTRATHECAL                  1
Name: TSVAL, dtype: int64

In [183]:
96 / 138

0.6956521739130435